# Rendu projet Neo4J

## Introduction
Ce travail a été réalisé par Baptiste Chezaubernard, Alix Ouedraogo et Alexis Aubrée.

Nous avons choisi de rendre un notebook, car ce format nous permet de mélanger la documentation du travail réalisé avec du code et de la visualisation.

Ci dessous, nous importons l'ensemble des librairies et fonctions nécessaires au bon fonctionnement de ce notebook.

In [62]:
%run python/utils.py

Imports réalisés


### Contexte
Ce projet a pour but d'utiliser Neo4j pour étudier des graphes.
Nous avons choisi d'utiliser les données de [spotify](https://zenodo.org/record/2594557#.XeYt25NKhqO).

Ces données représentent les interractions entre les utilisateurs, les chansons, les playlists et les artistes. Les chansons et artistes sont majoritairement du genre rock.

Une partie des données est présentées ci-dessous.

In [51]:
readHead("data/base.csv")

,user_id,artist_name,track_name,playlist_name
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010


Nous avons enrichi ces données à l'aide de l'API de spotify. Nous avons rajouté pour chaque artiste les artistes liés, la popularité, leurs genres musicaux.

Un extrait du fichier obtenu est présenté ci-dessous.

In [52]:
readHead("data/artists.csv")

,artist_name,artist_id,genres,popularity,related
0,Elvis Costello,2BGRfQgtzikz1pzAD0kaEn,"['art rock', 'dance rock', 'folk', 'folk rock'...",60.0,"['Elvis Costello & The Attractions', 'Squeeze'..."
1,Elvis Costello & The Attractions,4qmHkMxr6pTWh5Zo74odpH,"['art rock', 'comic', 'dance rock', 'folk rock...",59.0,"['Elvis Costello', 'Squeeze', 'Joe Jackson', '..."
2,Tiffany Page,1Q6EgKHZ8mt7Hm5bKa6DbG,[],4.0,"['Heidi Range', 'Alex Roots', 'Hepburn', 'Heav..."


Enfin, nous avons complété nos données à l'aide des données des concerts pour les artistes. Ces données ont été obtenue à l'aide du site [concertarchive](https://www.concertarchives.org/).

Une partie de ces données est présentée ci-dessous

In [58]:
readHead("data/concerts.csv", delimiter="|")

,artist,date,nom_concert,endroit,ville
0,Elvis Costello,"Mar 08, 2014",Elvis Costello / Elvis Costello & The Imposters,Caesar's Atlantic City,"Atlantic City, NJ"
1,Elvis Costello,"Jul 06, 2003",Elvis Costello & The Imposters / Imposters / E...,Taste of Chicago,"Chicago, IL"
2,Elvis Costello,"Nov 10, 2019",Elvis Costello & The Imposters / Elvis Costell...,The Mahaffey Theater,"St Petersburg, FL"


## 1. Imports des données

### Connexion à la base de données

In [63]:
DB_URI = "bolt://localhost:7687"
DB_USER = "neo4j"
DB_PASSWORD = "password"
driver = GraphDatabase.driver(DB_URI, auth=(DB_USER, DB_PASSWORD))

### Définition de la structure de la base

In [78]:
CONSTRAINTS=[
    "CREATE CONSTRAINT ON (u:User) ASSERT u.id IS UNIQUE;",
    "CREATE CONSTRAINT ON (s:Song) ASSERT s.track_name IS UNIQUE;",
    "CREATE CONSTRAINT ON (a:Artist) ASSERT a.artist_name IS UNIQUE;", 
    "CREATE CONSTRAINT ON (p:Playlist) ASSERT p.playlist_name IS UNIQUE; ",
    "CREATE CONSTRAINT ON (g:Genre) ASSERT g.genre_name IS UNIQUE;",
]
with driver.session() as session:
        for constraint in CONSTRAINTS:
            session.run(constraint)

## 2. Requetes